In [2]:
import os
import sys
import gdal
import ntpath
import argparse
import pandas as pd

In [3]:
# get one file
def get_file(folder):
    return (f for f in os.listdir(folder) if f.endswith('.' + 'tif'))

In [6]:
# get center for path of each requested folder
def get_center(trip_folder):
    
    flights = []
    # gets the trip folder and parses to each output file
    folders = next(os.walk(trip_folder))[1]
    
    # get center from each trip folder's flight
    for folder in folders:
        flight = []
        # first append site name
        flight.append(folder)
        
        # go to output
        folder = os.path.join(trip_folder, folder, "output", "")
        # check if output exists
        if not os.path.isdir(folder):
            continue
        
        # Prepare file list
        files = list(get_file(folder))
        # check if empty
        if len(files) == 0:
            continue
            
        # just get one 
        file = folder + files[0]

        # Vars
        lat = "Invalid File"
        long = ""

        # Analyze image file
        image_transform = size = []
        if file != "":
            image_info = gdal.Info(file, format="json")
            image_transform = image_info['geoTransform']
            size = image_info["size"]

        # Calculate centers
        width = image_transform[1] * size[0]
        height = image_transform[5] * size[1]
        long = image_transform[0] + 0.5 * width
        lat = image_transform[3] + 0.5 * height

        # Print file info
        name = ntpath.basename(file)
        print("{0:<40}{1:<20}{2:<20}".format(name, lat, long))
        
        # add lat and long
        flight.append(lat)
        flight.append(long)
        
        # store info
        flights.append(flight)

    # store info in dataframe
    cols = ["Site", "Latitude", "Longitude"]
    trip_coords = pd.DataFrame(flights, columns=cols)
    return trip_coords

In [15]:
trips = ["2017.07-mexico-ptosancarlos", "2018.05-mexico-ptosancarlos", "2018.07-mexico-lapaz", 
        "2018.07-mexico-ptosancarlos", "2019.05-mexico-skr"]
all_coords = pd.DataFrame(columns=["Site", "Latitude", "Longitude"])
# go through all trips and get their coordinates
for trip in trips:
    trip = os.path.join("/data/mangrove-data/master", trip, "")
    coords = get_center(trip)
    all_coords = pd.concat([all_coords, coords], ignore_index=True)

25-26-27_ortho_mult.tif                 24.797049370967102  -112.09486131261201 
17-18_ortho_mult.tif                    24.8024476017033    -112.114589921839   
lagoon2_ortho.tif                       24.79493284767799   -112.18143401557215 
lagoon1_ortho.tif                       24.791084198490985  -112.18776451479367 
10-11-12_ortho_mult.tif                 24.8010570421986    -112.110954609557   
3-4_ortho_mult.tif                      24.8034400473051    -112.10274302150799 
1-2_ortho.tif                           24.79364348413935   -112.11564702746895 
site-3-4-ortho.tif                      24.7979444788103    -112.09529453071049 
psc_2018-05_site12_120m_DEM.tif         24.785276643722103  -112.085501893892   
psc_2018-05_site01-04_sat_RGBI.tif      2744545.5           384873.0            
psc_2018-05_site13-14_120m_DEM.tif      24.7825155554456    -112.094090767036   
site-9-ortho.tif                        24.7936792871419    -112.11595903032    
psc_2018-05_site03_120m_RGB_

In [19]:
all_coords.to_csv('/data/mangrove-data/Kathy/biomass_project/scripts/get_gps/all_sites_gps.csv', index=False)